<a href="https://colab.research.google.com/github/sudeep93/CE888/blob/main/CE888project/TweetClassifier_2203303.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing the transformers library

In [ ]:
!pip install transformers
!pip3 install emoji

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### download punkt and stopwords

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')


### Import Required Librearies

In [ ]:
import numpy as np
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import nltk
import imblearn
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import emoji
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout, Input
from tensorflow.keras import regularizers
import pickle
import string
import re
import matplotlib.pyplot as plt
from transformers import BertTokenizer, AutoModel, AutoTokenizer,TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import transformers as ppb
import warnings
import keras
warnings.filterwarnings('ignore')

## Function to Clean the Text tweet Data

In [ ]:
pd.set_option('display.max_colwidth',1000)
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
words = set(nltk.corpus.words.words())

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

# clean the text dataframe
def text_cleaning(text):
  text = text.lower()
        
  text = re.sub(r'http.?:\/\/\S+','',text) #remove links
  text = emoji.demojize(text, delimiters=("", "")) 
  # text = text.encode("ascii", errors="ignore").decode()
  text 
  text=re.sub(r'@[A-Za-z0-9]+','',text) #removes mentions
  text=re.sub(r'#','',text)#remove hashtag
  text = re.sub(r'RT[\s]','',text) #remove RT
  text = re.sub(r'http?:\/\/\S+','',text) #remove links
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"isn’t ", "is not", text)
  text = re.sub(r"y'all", "you all", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"here's", "here is", text)
  text = re.sub(r"that's", "that is", text)        
  text = re.sub(r"what's", "what is", text)
  text = re.sub(r"where's", "where is", text) 
  text = re.sub(r"\’s", " is", text)
  text = re.sub(r"\'ll", " will", text)  
  text = re.sub(r"they'll", "they will", text)  
  text = re.sub(r"\'ve", " have", text)  
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"don't", "do not", text)
  text = re.sub(r"did't", "did not", text)
  text = re.sub(r"did’t", "did not", text)
  text = re.sub(r"can't", "can not", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"couldn't", "could not", text)
  text = re.sub(r"have't", "have not", text)
  text = re.sub(r"qt", "", text)
  text = re.sub(r"_", " ", text)
  text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-]", " ", text)
  text=re.sub(r'\'','',text)
  text=re.sub(r'\"','',text)
  text=re.sub(r'\“','',text)
  text=re.sub(r'\”','',text)
  tokens = word_tokenize(text)
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  words1 = [word for word in stripped if word.isalpha()]
  stop_words = set(stopwords.words("english"))
  stop_words.discard("not")

  words1 = [w for w in words1 if not w in stop_words]
  words1 = ' '.join(words1)
  # print(words1)
  # words1 = " ".join(w for w in nltk.wordpunct_tokenize(words1) if w.lower() in words or not w.isalpha())
  # words1 = [w for w in words1 if  w in words.words()]
  # final = ' '.join(words1)
  final=[lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(words1)]
  final = ' '.join(final)
  return words1

### Apply cleaning function on Train  and test dataset and importing dataset

In [ ]:
def dataset_cleaning(path_tweet,path_label):
  # load data
  print("cleaing tweets on path",path_tweet)
  col_names=['tweets', 'labels'] 
  df = pd.read_csv(path_tweet, sep ="\n",names=col_names,header=None)
    # load data label
  labels = pd.read_csv(path_label, sep ="\n",header=None)
  df['labels']=labels
  # Cleaning data 
  df['tweets']=  df['tweets'].apply(text_cleaning)

  print("Finished cleaing tweets on path",path_tweet,"\n \n")
  return df

hate_train=dataset_cleaning('/content/drive/MyDrive/datasets/hate/train_text.txt','/content/drive/MyDrive/datasets/hate/train_labels.txt')
hate_test=dataset_cleaning('/content/drive/MyDrive/datasets/hate/test_text.txt','/content/drive/MyDrive/datasets/hate/test_labels.txt')
hate_val=dataset_cleaning('/content/drive/MyDrive/datasets/hate/val_text.txt','/content/drive/MyDrive/datasets/hate/val_labels.txt')

irony_train=dataset_cleaning('/content/drive/MyDrive/datasets/irony/train_text.txt','/content/drive/MyDrive/datasets/irony/train_labels.txt')
irony_test=dataset_cleaning('/content/drive/MyDrive/datasets/irony/test_text.txt','/content/drive/MyDrive/datasets/irony/test_labels.txt')
irony_val=dataset_cleaning('/content/drive/MyDrive/datasets/irony/val_text.txt','/content/drive/MyDrive/datasets/irony/val_labels.txt')


offensive_train=dataset_cleaning('/content/drive/MyDrive/datasets/offensive/train_text.txt','/content/drive/MyDrive/datasets/offensive/train_labels.txt')
offensive_test=dataset_cleaning('/content/drive/MyDrive/datasets/offensive/test_text.txt','/content/drive/MyDrive/datasets/offensive/test_labels.txt')
offensive_val=dataset_cleaning('/content/drive/MyDrive/datasets/offensive/val_text.txt','/content/drive/MyDrive/datasets/offensive/val_labels.txt')





In [ ]:
irony_train['tweets'].tail(10)


##Label Distribution

### Hate label distribution

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="labels", data=hate_train).set_title('Hate Labels')


### Irony label distribution

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="labels", data=irony_train).set_title('Irony Labels')


### Offensive label distribution

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="labels", data=offensive_train).set_title('Offensive Labels')


### Frequency of the words visaulization test set in Hate,Irony and Offensive

In [ ]:
majorwords=' '.join([twt for twt in hate_train['tweets']])
majorwords=WordCloud(width=1000,height=500,random_state=32,max_font_size=130).generate(majorwords)
# plt.imshow(majorwords,interpolation='bilinear')
plt.figure(figsize=(15,8))
plt.imshow(majorwords)
plt.axis("off")

In [ ]:
majorwords=' '.join([twt for twt in irony_train['tweets']])
majorwords=WordCloud(width=1000,height=500,random_state=32,max_font_size=130).generate(majorwords)
# plt.imshow(majorwords,interpolation='bilinear')
plt.figure(figsize=(15,8))
plt.imshow(majorwords)
plt.axis("off")

In [ ]:
majorwords=' '.join([twt for twt in offensive_train['tweets']])
majorwords=WordCloud(width=1000,height=500,random_state=32,max_font_size=130).generate(majorwords)
# plt.imshow(majorwords,interpolation='bilinear')
plt.figure(figsize=(15,8))
plt.imshow(majorwords)
plt.axis("off")

In [ ]:
irony_test

##Loading the Pre-trained DistilBERT model
Let's now load a pre-trained DistilBERT model. 

In [ ]:

# Loading DistilBERT Tokenizer and the DistilBERT model
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

## Check the MAX_length of Entire dataset

In [ ]:


def max_length(df_train,df_val,df_test):
  b=[]
  test=pd.concat([df_train['tweets'], df_val['tweets'],df_test['tweets']], ignore_index=True, sort=True)
  for i in test:
    a=dbert_tokenizer.tokenize(i)
    b.append(len(a))
  df_t = pd.DataFrame(b, columns=['A'])
  return df_t.nlargest(5,'A', keep='all')




  



## MAX LENGHT OF HATE IRONY AND OFFENSIVE FOR PADDING

In [ ]:
max_len_hate=max_length(hate_test,hate_train,hate_val)
max_len_irony=max_length(irony_test,irony_train,irony_val)
max_len_offensive=max_length(offensive_test,offensive_train,offensive_val)
print("FOR HATE",max_len_hate,"\n")
print("FOR IRONY",max_len_irony,"\n")
print("FOR OFFENSIVE",max_len_offensive,"\n")

max_len=100

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format DistilBERT is comfortable with.

## Create NN Model for classification using Keras

In [ ]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout= Dropout(0.5)(dense)
    pred = Dense(2, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model

## Create Three models for Three datasets ie hate irony and offensive

In [ ]:
model_hate=create_model()
model_irony=create_model()
model_offensive=create_model()

##FUCNTION

In [ ]:
def distil_bert_process(df,max_len):
# Preparing input for the model

  sentences=df['tweets']
  dbert_inp=dbert_tokenizer.encode_plus(sentences[0],add_special_tokens = True,max_length =max_len,pad_to_max_length = True,truncation=True)
  dbert_inp
  print("Tokenization In-process")
  id_inp=np.asarray(dbert_inp['input_ids'])
  mask_inp=np.asarray(dbert_inp['attention_mask'])
  out=dbert_model([id_inp.reshape(1,-1),mask_inp.reshape(1,-1)])
  type(out),out
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])

  input_ids=np.array(input_ids) 
  attention_masks=np.array(attention_masks)
  labels=np.array(df['labels'])
  print("Tokenization finished")
  return input_ids,attention_masks,labels
    

In [ ]:
def training(df_train,df_val,model,no_epochs,max_len,save_path):
# train dataset dbert process
  id_train,att_train,label_train=distil_bert_process(df_train,max_len)

# val dataset dbert process 
  id_val,att_val,label_val=distil_bert_process(df_val,max_len)

  print("Model Builing in process")

  log_dir='dbert_model'
  model_save_path='./{}.h5'.format(save_path)

  callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  print("Compiling Model")
  model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  print("Fitting Model")
  history=model.fit([id_train,att_train],label_train,batch_size=16,epochs=no_epochs,validation_data=([id_val,att_val],label_val),callbacks=callbacks)
  print("Fitting finished")
  return history


###  Training and Prediction

#### Hate Dataset training and predicting and evaluation 

In [ ]:
# Train data
training_hate=training(hate_train,hate_val,model_hate,8,100,"hate_model")


In [ ]:
# Plot results
acc = training_hate.history['accuracy']
val_acc = training_hate.history['val_accuracy']
loss = training_hate.history['loss']
val_loss = training_hate.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
id_hate_test,att_hate_test,label_hate_test=distil_bert_process(hate_test,100)


In [ ]:
pred_labels_hate=model_hate.predict([id_hate_test,att_hate_test])

In [ ]:
y_pred_bool_hate = np.argmax(pred_labels_hate, axis=1)

In [ ]:
# Classification Report for HATE DATASET
print('Classification Report for HATE DATASET')
print(classification_report(label_hate_test,y_pred_bool_hate))
print(accuracy_score(label_hate_test,y_pred_bool_hate))


#### Irony Dataset training and predicting and evaluation 

In [ ]:
# Train data
training_irony=training(irony_train,irony_val,model_irony,20,100,"irony_model")


In [ ]:
# Plot results
acc = training_irony.history['accuracy']
val_acc = training_irony.history['val_accuracy']
loss = training_irony.history['loss']
val_loss = training_irony.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
id_irony_test,att_irony_test,label_irony_test=distil_bert_process(irony_test,100)


In [ ]:
pred_labels_irony=model_irony.predict([id_irony_test,att_irony_test])

In [ ]:
y_pred_bool_irony = np.argmax(pred_labels_irony, axis=1)

In [ ]:
# Classification Report for IRONY Dataset
print('Classification Report for IRONY Dataset')
print(classification_report(label_irony_test,y_pred_bool_irony))

#### Offensive Dataset training and predicting and evaluation 

In [ ]:
# Train data
training_offensive=training(offensive_train,offensive_val,model_offensive,6,100,"offensive_model")


In [ ]:
# Plot results
acc = training_offensive.history['accuracy']
val_acc = training_offensive.history['val_accuracy']
loss = training_offensive.history['loss']
val_loss = training_offensive.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
id_offensive_test,att_offensive_test,label_offensive_test=distil_bert_process(offensive_test,100)


In [ ]:
pred_labels_offensive=model_offensive.predict([id_offensive_test,att_offensive_test])

In [ ]:
y_pred_bool_offensive = np.argmax(pred_labels_offensive, axis=1)

In [ ]:
# Classification Report for OFFENSIVE Dataset
print('Classification Report for OFFENSIVE Dataset')
print(classification_report(label_offensive_test,y_pred_bool_offensive))

